# Import necessary Libraries

In [ ]:
#External package need to install
!pip install apyori

In [ ]:
from timeit import default_timer
start = default_timer()

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')

In [ ]:
import pandas as pd
tag = pd.read_csv("../input/movielens-20m-dataset/tag.csv")
movies = pd.read_csv("../input/movielens-20m-dataset/movie.csv")
rating = pd.read_csv("../input/movielens-20m-dataset/rating.csv")
link = pd.read_csv("../input/movielens-20m-dataset/link.csv")
#genome_tags = pd.read_csv("../input/movielens-20m-dataset/genome_tags.csv")
#genome_scores = pd.read_csv("../input/movielens-20m-dataset/genome_scores.csv")

# Exploratory Data Aanalysis

In [ ]:
tag.head(15)

In [ ]:
tag.describe()

In [ ]:
tag.info()

In [ ]:
tag["tag"].unique()

In [ ]:
tag["userId"].unique()

In [ ]:
tag["movieId"].unique()

In [ ]:
tag["timestamp"].unique()

In [ ]:
tag.isnull().sum()

In [ ]:
tag.isna().sum()

In [ ]:
tag.shape

In [ ]:
tag.dtypes

In [ ]:
movies.head(15)

In [ ]:
movies.describe()

In [ ]:
movies.info()

In [ ]:
movies["genres"].unique()

In [ ]:
movies["title"].unique()

In [ ]:
movies.isnull().sum()

In [ ]:
movies.isna().sum()

In [ ]:
movies.dtypes

In [ ]:
rating.head()

In [ ]:
rating.isnull().sum()

In [ ]:
result = pd.merge(movies[['movieId','title']],rating,on='movieId')


In [ ]:
result.head()

In [ ]:
result.drop('timestamp',inplace=True,axis=1)

In [ ]:
len(result)


In [ ]:
result1 = result[result['rating'] > 3.5]
non_favourite = result[result['rating'] <= 3.5]

In [ ]:
len(result1)

In [ ]:
non_favourite = non_favourite[['userId', 'movieId', 'title', 'rating']]


In [ ]:
non_favourite.head()

In [ ]:
a = len(non_favourite)

In [ ]:
non_fav1 = non_favourite[:a//2]
non_fav2 = non_favourite[a//2:]

In [ ]:
Non_Favourite1 = non_fav1.to_csv('non_favourite1.csv', index=False)
Non_Favourite2 = non_fav2.to_csv('non_favourite2.csv', index=False)


In [ ]:
result1.head()

In [ ]:
len(result1)

In [ ]:
result1 = result1[:1000000]


In [ ]:
basket = pd.pivot_table(data=result1,index='userId',columns='title',values='rating',fill_value=0)
                        

In [ ]:
def convert_into_binary(x):
    if x > 0:
        return 1
    else:
        return 0

In [ ]:
basket_sets = basket.applymap(convert_into_binary)

In [ ]:
basket_sets.head()

In [ ]:
from apyori import apriori

In [ ]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules


In [ ]:
#call apriori function and pass minimum support here we are passing 7%. means 7 times in total number of transaction that item was present.
frequent_itemsets = apriori(basket_sets, min_support=0.07, use_colnames=True)

In [ ]:
#it will generate frequent itemsets using two step approch
frequent_itemsets

In [ ]:
rules_mlxtend = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

In [ ]:
rules_mlxtend.head()

In [ ]:
# as based business use case we can sort based on confidance and lift.
fav_movies = rules_mlxtend[ (rules_mlxtend['lift'] >= 1.5) & (rules_mlxtend['confidence'] >= 0.5) ]

In [ ]:
fav_movies = pd.DataFrame(fav_movies)

In [ ]:
len(fav_movies) 

**Let's fetch the correpsonding association rules generated via Apriori algorithm for same movie and sort them in descending order of lift**

In [ ]:
all_antecedents = [list(x) for x in fav_movies.antecedents.values]
len(all_antecedents)

In [ ]:
all_antecedents = [list(x) for x in fav_movies.antecedents.values]
desired_indices = []
for i in range (len(all_antecedents)) : #liste de liste
    for j in range (len(all_antecedents[i])) : 
        if  all_antecedents[i][j] == "Pulp Fiction (1994)" : 
            desired_indices.append(i)
            

apriori_recommendations=fav_movies.iloc[desired_indices,].sort_values(by=['lift'],ascending=False)
apriori_recommendations.head()
            

In [ ]:
#TOP 5
apriori_recommendations_list = [list(x) for x in apriori_recommendations['consequents'].values]
print("Apriori Recommendations for basket containing : Pulp fiction\n")
for i in range(5):
    print("{0}: {1} with confidence of {2}".format(i+1,apriori_recommendations_list[i],apriori_recommendations.iloc[i,5]))

**code for 1 movie not basket**

In [ ]:
#all_antecedents = [list(x) for x in fav_movies.antecedents.values]
#desired_indices = []
#for i in range (len(all_antecedents)) : #liste de liste
#    if (len(all_antecedents[i]) == 1) and (all_antecedents[i][0] == "Pulp Fiction (1994)") : 
#          desired_indices.append(i)
            

#apriori_recommendations=fav_movies.iloc[desired_indices,].sort_values(by=['lift'],ascending=False)
#apriori_recommendations.head()
            


In [ ]:
s = 0
for x in basket_sets.iloc[200] :
    if  x == 1 :
        s+=1

        

        
        

In [ ]:
basket_sets.iloc[175:190]

In [ ]:
fav_movies

In [ ]:
movies_list = basket_sets.columns.tolist()
movies_list[1:5]

In [ ]:
len(movies_list)

In [ ]:
def recommendations(movie_list,movie_name) : 
    all_antecedents = [list(x) for x in movie_list.antecedents.values] #fav_movies
    desired_indices = []
    for i in range (len(all_antecedents)) : #liste de liste
        for j in range (len(all_antecedents[i])) : 
            if  all_antecedents[i][j] == movie_name : 
                desired_indices.append(i)
            
    
    apriori_recommendations=movie_list.iloc[desired_indices,].sort_values(by=['confidence'],ascending=False)
    return apriori_recommendations

In [ ]:
a = recommendations(fav_movies,"Pulp Fiction (1994)")
b = a.consequents
b = pd.DataFrame(b)
b

#b = a.consequents
b.iloc[4] # b = consequent

In [ ]:
def recommendation_system(basket_sets,movie_list,user_number) : 
    rec_list = []
    rec = []
    new_list = []
    a = []
    result = set()
    #final_list = []
    #for i in range (len(basket_sets)) : 
    for j in range (len(movies_list)) :
        if basket_sets.iloc[user_number,j] == 1 :
            rated_movie = movies_list[j]
            rec = recommendations(fav_movies,rated_movie) #data frame
            #consequents = rec.consequents.values
            #rec_list.append(consequents)
            rec_list = [list(x) for x in rec.consequents.values]
     
    
            
    for k in range(len(rec_list)) : 
        result.add(rec_list[k][0])
    

    return  result
            #apriori_recommendations_list = [list(x) for x in rec['consequents'].values]
            #rec_list.append( apriori_recommendations_list)
        
 #fonction for 1 user 

In [ ]:
el_aflem = recommendation_system(basket_sets,movies_list,198)
el_aflem

#user 200 el aflem = 0 => CF ?


In [ ]:
#el_aflem = pd.DataFrame(el_aflem)

In [ ]:
#plotting output in a graph plot.

import networkx as nx
import matplotlib.pyplot as plt  

def draw_graph(rules, rules_to_show):
    G1 = nx.DiGraph()
    color_map=[]
    N = 50
    colors = np.random.rand(N)    
    strs=['R0', 'R1', 'R2', 'R3', 'R4', 'R5', 'R6', 'R7', 'R8', 'R9', 'R10', 'R11']

    for i in range(rules_to_show):
        G1.add_nodes_from(["R"+str(i)])
        for a in rules.iloc[i]['antecedents']:
            G1.add_nodes_from([a])
            G1.add_edge(a, "R"+str(i), color=colors[i] , weight = 2)
        for c in rules.iloc[i]['consequents']:
            G1.add_nodes_from([c])
            G1.add_edge("R"+str(i), c, color=colors[i],  weight=2)

    for node in G1:
        found_a_string = False
        for item in strs: 
            if node==item:
                found_a_string = True
        if found_a_string:
            color_map.append('yellow')
        else:
            color_map.append('green')       

    edges = G1.edges()
    colors = [G1[u][v]['color'] for u,v in edges]
    weights = [G1[u][v]['weight'] for u,v in edges]

    pos = nx.spring_layout(G1, k=16, scale=1)
    nx.draw(G1, pos, edges=edges, node_color = color_map, edge_color=colors, width=weights, font_size=16, 
            with_labels=False)            

    for p in pos:  # raise text positions
        pos[p][1] += 0.07
        nx.draw_networkx_labels(G1, pos)
        plt.show()

draw_graph (rules_mlxtend, 10) 